# Lesson 2: Advanced Agentic AI

## Grounding Foundation Models & Multi-Agent Committees

In Lesson 1, you learned that LLMs are powerful pattern predictors but have limitations:
- They can't access real-time information
- They don't know about YOUR specific data
- They work in isolation

Today, we'll solve these problems by:
1. **Grounding** LLMs with your own knowledge base
2. **Coordinating** multiple agents to solve complex tasks

# **Guiding Questions:**
1. How can we make LLMs answer questions about information they've never seen?
2. Can multiple AI agents work together better than one?

---

# Part 1: Grounding Foundation Models

## What Does "Grounding" Mean?

**Grounding** means connecting an LLM's responses to specific, verifiable sources of information.

### The Problem:
- LLMs only know what was in their training data (cutoff: early 2024 for most models)
- They hallucinate when they don't know something
- They can't access YOUR documents, databases, or private information

### The Solution: RAG (Retrieval-Augmented Generation)
**RAG** = Retrieve relevant information → Augment the prompt → Generate grounded responses

Think of it like an open-book exam vs. a closed-book exam!

## Setup: Import Libraries

Let's set up our environment for building a grounded agent.

In [1]:
import os
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '1'
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import asyncio
from dotenv import load_dotenv

from fairlib.utils.document_processor import DocumentProcessor

from fairlib import (
    settings,
    Message,
    HuggingFaceAdapter,
    ToolRegistry,
    ToolExecutor,
    WorkingMemory,
    LongTermMemory,
    ChromaDBVectorStore,
    ReActPlanner,
    SimpleAgent,
    SentenceTransformerEmbedder,
    SimpleRetriever,
    KnowledgeBaseQueryTool  # <-- Using the official framework tool
)

# ChromaDB for vector storage
try:
    import chromadb
    CHROMADB_AVAILABLE = True
except ImportError:
    print("Warning: chromadb not installed. Install with: pip install chromadb")
    CHROMADB_AVAILABLE = False

# Load environment variables
load_dotenv()
token = os.getenv("HUGGING_FACE_HUB_TOKEN")

if not token:
    print("Warning: HUGGING_FACE_HUB_TOKEN not found in .env file!")
else:
    print("Token loaded successfully!")

Token loaded successfully!


## Experiment 1: The Hallucination Problem

Let's see what happens when we ask an LLM about information it doesn't have.

**Try this**: Google "UCCS Kraemer Library hours" and see what the actual hours are. Then compare to what the LLM says!

In [2]:
# Load a language model
print("Loading language model...")
llm = HuggingFaceAdapter(
    model_name="dolphin3-qwen25-3b", 
    auth_token=token,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    max_new_tokens=512,
    repetition_penalty=1.1
)
print("Model loaded!")

Loading language model...
🔧 Loading HuggingFace model: cognitivecomputations/Dolphin3.0-Qwen2.5-3b (quantized=False, stream=False)


Device set to use mps


Model loaded!


In [3]:
# Ask about UCCS library hours
question = "What time does the Kraemer Family Library at UCCS close on Friday nights?"

messages = [Message(role="user", content=question)]
response = llm.invoke(messages)

print(f"Question: {question}")
print(f"\nLLM Response:\n{response.content}")
print("\nWARNING: This information might be completely made up!")
print("\nGoogle 'What time does the Kraemer Family Library at UCCS close on Friday nights?' to verify if this is correct!")

Question: What time does the Kraemer Family Library at UCCS close on Friday nights?

LLM Response:
The Kraemer Family Library at UCCS typically closes around 5:00 PM on Fridays, but it's always best to check their official website or contact them directly for the most accurate information as hours can vary depending on the day of the week and other factors.


Google 'What time does the Kraemer Family Library at UCCS close on Friday nights?' to verify if this is correct!


### Reflection Question 1

**Did the LLM give you the right answer? Did it admit it doesn't know, or did it make something up?**

This is the hallucination problem: LLMs try to answer even when they don't have the information!

## Building a Knowledge Base with Real Documents

Instead of relying on the LLM's training data, let's create actual documents with UCCS information.

In a real system, you'd have PDFs, Word docs, or web pages. For this demo, we'll create text files.

In [4]:
# Create a directory for our knowledge base documents
kb_directory = "./uccs_knowledge_base"
os.makedirs(kb_directory, exist_ok=True)

# Document 1: Library Hours
library_hours_doc = """Kraemer Family Library Hours (UCCS)

Fall/Spring Semester Regular Hours:
- Monday - Thursday: 7:30 AM - 11:00 PM
- Friday: 7:30 AM - 6:00 PM
- Saturday: 10:00 AM - 6:00 PM
- Sunday: 12:00 PM - 11:00 PM

Finals Week Extended Hours:
- Sunday - Thursday: 7:30 AM - 2:00 AM
- Friday: 7:30 AM - 6:00 PM
- Saturday: 10:00 AM - 6:00 PM

Summer and Break Hours:
- Monday - Friday: 8:00 AM - 5:00 PM
- Closed on weekends

Note: Hours may vary during holidays. Always check library.uccs.edu for the most current information.
"""

# Document 2: Degree Requirements
degree_requirements_doc = """UCCS Computer Science B.S. Degree Requirements

Total Credit Hours Required: 120

Core Computer Science Courses: 48 credit hours
- CS 1030, 1050, 1150, 1200, 2060, 2100, 2400
- CS 3100, 3150, 3300, 3400, 3500, 4200
- CS 4310 (Senior Design I) and CS 4320 (Senior Design II)

Mathematics Requirements: 16 credit hours
- MATH 1310, 1320, 2130, 2420

General Education: 35 credit hours
- UCCS Core and Compass Curriculum requirements

Electives and Free Choice: 21 credit hours

Additional Requirements:
- Minimum 2.0 GPA in major courses
- At least 30 credit hours must be upper-division (3000-4000 level)
- Senior design capstone project required
- Complete at least 45 credit hours at UCCS
"""

# Document 3: Campus Resources
campus_resources_doc = """UCCS Student Resources and Services

Writing Center:
- Location: Columbine Hall, Room 108
- Services: Free writing tutoring for all UCCS students
- Walk-in hours: Monday-Friday, 9:00 AM - 5:00 PM
- Online appointments available via Zoom
- Website: writingcenter.uccs.edu

Wellness Center:
- Location: University Center, Second Floor
- Services: Medical care, counseling, health education
- Phone: 719-255-4444
- Hours: Monday-Friday, 8:00 AM - 5:00 PM
- Crisis support available 24/7

Career Center:
- Location: Cragmor Hall, Room 111
- Services: Resume reviews, interview preparation, job search assistance
- Handshake platform for job/internship postings
- Career fairs held each semester (Fall and Spring)
- One-on-one advising appointments available

Math Learning Center:
- Location: Engineering Building, Room 105
- Free tutoring for math courses up to Calculus II
- Walk-in hours: Monday-Thursday, 10:00 AM - 6:00 PM

Computer Science Tutoring Lab:
- Location: Engineering Building, Room 207
- Free tutoring for CS 1030, 1050, 1150, 1200, 2060
- Hours: Monday-Friday, 12:00 PM - 5:00 PM
"""

# Save documents to files
with open(os.path.join(kb_directory, "library_hours.txt"), "w") as f:
    f.write(library_hours_doc)

with open(os.path.join(kb_directory, "degree_requirements.txt"), "w") as f:
    f.write(degree_requirements_doc)

with open(os.path.join(kb_directory, "campus_resources.txt"), "w") as f:
    f.write(campus_resources_doc)

print("Knowledge base documents created!")
print(f"Saved to: {kb_directory}")
print(f"Documents: library_hours.txt, degree_requirements.txt, campus_resources.txt")

Knowledge base documents created!
Saved to: ./uccs_knowledge_base
Documents: library_hours.txt, degree_requirements.txt, campus_resources.txt


## Processing Documents with FAIR_LLM

Now we'll use FAIR_LLM's `DocumentProcessor` to load and chunk these documents.

This is the same process you'd use for PDFs, Word docs, PowerPoints, etc.!

In [5]:
# Initialize the document processor
doc_processor = DocumentProcessor(config={
    "files_directory": kb_directory,
    "max_chunk_chars": 1000,
    "supported_extensions": {".txt", ".pdf", ".docx"}
})

print("Document processor initialized!")

Document processor initialized!


In [6]:
# Load all documents from the folder
print("Loading and chunking documents...\n")
documents = doc_processor.load_documents_from_folder()

print(f"\nLoaded {len(documents)} document chunks")
print(f"\nExample chunk:")
print(f"Content: {documents[0].page_content[:200]}...")
print(f"\nMetadata: {documents[0].metadata}")

Loading and chunking documents...


Loaded 8 document chunks

Example chunk:
Content: Kraemer Family Library Hours (UCCS) Fall/Spring Semester Regular Hours:
- Monday - Thursday: 7:30 AM - 11:00 PM
- Friday: 7:30 AM - 6:00 PM
- Saturday: 10:00 AM - 6:00 PM
- Sunday: 12:00 PM - 11:00 PM...

Metadata: {'filename': 'library_hours.txt', 'type': '.txt', 'segment_index': 0, 'chunk_index': 0, 'source': 'library_hours.txt'}


## Building the RAG Architecture

Now we'll build the proper RAG pipeline with the core FAIR_LLM components:

1. **Embedder**: Converts text into numerical vectors
2. **Vector Store**: Stores embeddings and enables similarity search  
3. **Long-Term Memory**: Wraps the vector store
4. **Retriever**: Queries the vector store to find relevant documents

**How it works:**
1. Documents are converted into vector embeddings (numerical representations)
2. Embeddings are stored in a vector database (ChromaDB)
3. When you query, the query is converted to a vector
4. Vector store finds the most similar document vectors (semantic search)
5. Retriever returns the most relevant chunks

In [7]:
# Check if ChromaDB is available
if not CHROMADB_AVAILABLE:
    print("ERROR: ChromaDB is required for this section.")
    print("Install it with: pip install chromadb")
    print("\nSkipping RAG setup...")
else:
    print("ChromaDB available")

ChromaDB available


In [8]:
# Step 1: Create the embedder
print("Step 1: Creating embedder...")
embedder = SentenceTransformerEmbedder(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # Fast, efficient model
)
print("✓ Embedder created")

Step 1: Creating embedder...
✅ SentenceTransformerEmbedder initialized with model: sentence-transformers/all-MiniLM-L6-v2
✓ Embedder created


In [9]:
# Step 2 & 3: Create vector store and add documents (combined to prevent errors)
print("\nStep 2 & 3: Creating vector store and adding documents...")

# Create a fresh ChromaDB client
chroma_client = chromadb.Client()

# Delete collection if it exists (prevents duplicate IDs on reruns)
try:
    chroma_client.delete_collection(name="uccs_knowledge_base")
    print("  (Cleared existing collection)")
except:
    pass

# Create fresh vector store
vector_store = ChromaDBVectorStore(
    embedder=embedder,
    client=chroma_client,
    collection_name="uccs_knowledge_base"
)
print("✓ Vector store created")

# Extract text content from Document objects
document_texts = [doc.page_content for doc in documents]

# CRITICAL: Remove duplicate documents (ChromaDB uses hash(doc) as ID internally)
unique_texts = []
seen = set()
for text in document_texts:
    if text not in seen:
        seen.add(text)
        unique_texts.append(text)

print(f"  Total chunks: {len(document_texts)}, Unique chunks: {len(unique_texts)}")

# Add to vector store (no ids parameter - ChromaDB generates them internally)
vector_store.add_documents(unique_texts)

print(f"✓ Added {len(unique_texts)} document chunks to vector store")


Step 2 & 3: Creating vector store and adding documents...
✓ Vector store created
  Total chunks: 8, Unique chunks: 5
✓ Added 5 document chunks to vector store


In [10]:
# Step 4: Create Long-Term Memory (wraps the vector store)
print("\nStep 4: Creating long-term memory...")
long_term_memory = LongTermMemory(vector_store)
print("✓ Long-term memory created")


Step 4: Creating long-term memory...
✓ Long-term memory created


In [11]:
# Step 5: Create the retriever
print("\nStep 5: Creating retriever...")
retriever = SimpleRetriever(vector_store)
print("✓ Retriever created and ready!")
print("\n" + "="*60)
print("RAG PIPELINE COMPLETE")
print("="*60)


Step 5: Creating retriever...
✅ SimpleRetriever initialized.
✓ Retriever created and ready!

RAG PIPELINE COMPLETE


### Test the Retriever Directly

Let's see how semantic search works!

In [12]:
# Test retrieval
test_query = "library hours friday"
results = retriever.retrieve(query=test_query, top_k=2)

print(f"Query: '{test_query}'")
print(f"\nTop {len(results)} most relevant chunks:\n")

for i, result in enumerate(results, 1):
    print(f"--- Result {i} ---")
    # Handle both string and Document results
    if isinstance(result, str):
        print(f"Content: {result[:300]}...\n")
    else:  # Document object
        if hasattr(result, 'metadata'):
            print(f"Source: {result.metadata.get('source', 'unknown')}")
        print(f"Content: {result.page_content[:300] if hasattr(result, 'page_content') else str(result)[:300]}...\n")

Query: 'library hours friday'

Top 2 most relevant chunks:

--- Result 1 ---
Content: Kraemer Family Library Hours (UCCS) Fall/Spring Semester Regular Hours:
- Monday - Thursday: 7:30 AM - 11:00 PM
- Friday: 7:30 AM - 6:00 PM
- Saturday: 10:00 AM - 6:00 PM
- Sunday: 12:00 PM - 11:00 PM Finals Week Extended Hours:
- Sunday - Thursday: 7:30 AM - 2:00 AM
- Friday: 7:30 AM - 6:00 PM
- Sa...

--- Result 2 ---
Content: Kraemer Family Library Hours (UCCS) Fall/Spring Semester Regular Hours:
- Monday - Thursday: 7:30 AM - 11:00 PM
- Friday: 7:30 AM - 6:00 PM
- Saturday: 10:00 AM - 6:00 PM
- Sunday: 12:00 PM - 11:00 PM Finals Week Extended Hours:
- Sunday - Thursday: 7:30 AM - 2:00 AM
- Friday: 7:30 AM - 6:00 PM
- Sa...



## Building a Grounded Agent with RAG

Now we'll create an agent that uses the `KnowledgeBaseQueryTool` to ground its responses!

In [13]:
# Create a separate LLM instance for the agent
agent_llm = llm

# Create the knowledge base query tool
knowledge_tool = KnowledgeBaseQueryTool(retriever=retriever)

# Register the tool
tool_registry = ToolRegistry()
tool_registry.register_tool(knowledge_tool)

# Create executor, memory, and planner
executor = ToolExecutor(tool_registry)
memory = WorkingMemory()
planner = ReActPlanner(agent_llm, tool_registry)

# Assemble the agent
grounded_agent = SimpleAgent(
    llm=agent_llm,
    planner=planner,
    tool_executor=executor,
    memory=memory,
    max_steps=5
)

grounded_agent.role_description = (
    "You are a UCCS student assistant. "
    "Your job is to answer student questions about UCCS accurately. "
    "ALWAYS search the knowledge base before answering - never make up information. "
    "Use the course_knowledge_query tool to find relevant information, then base your answer on those documents. "
    "If the information isn't in the knowledge base, say so and suggest where students can find more information."
)

print("✓ Grounded agent created successfully!")
print("✓ Agent has access to UCCS knowledge base via RAG")

Registering tool: course_knowledge_query
✓ Grounded agent created successfully!
✓ Agent has access to UCCS knowledge base via RAG


## Testing the Grounded Agent

Let's ask the same question - but now the agent has access to real documents!

In [14]:
async def test_grounded_agent(question):
    print(f"Question: {question}")
    print("\nAgent thinking...\n")
    
    response = await grounded_agent.arun(question)
    
    print(f"\n{'='*60}")
    print("Agent Response:")
    print(f"{'='*60}")
    print(response)
    print(f"{'='*60}\n")
    return response

In [ ]:
# Test 1: Question about library hours (the one that was hallucinated before!)
await test_grounded_agent("What time does the Kraemer Family Library at UCCS close on Friday nights?")

In [ ]:
# Test 2: Question about degree requirements
await test_grounded_agent("How many credit hours are required to graduate with a Computer Science degree from UCCS?")

In [ ]:
# Test 3: Question about campus resources
await test_grounded_agent("Where can I get help with my resume at UCCS?")

In [ ]:
# Test 4: Question about something NOT in the documents
await test_grounded_agent("What is the tuition cost for in-state students at UCCS?")

### Reflection Question 2

**What's different between the pure LLM and the grounded agent?**

Notice:
- The grounded agent **searches documents first** before answering
- Answers are **based on actual source text** (you can verify!)
- When info isn't available, it **admits it and suggests alternatives** (no hallucination!)
- You can **trace where the information came from**

This is **Retrieval-Augmented Generation (RAG)** in action!

## Understanding the RAG Pipeline

### Without Grounding (Pure LLM):
```
User: "What's the library closing time on Friday?"
  ↓
LLM: [Predicts based on general patterns]
  ↓
LLM: "Most libraries close around 10 PM on Fridays..." (HALLUCINATION!)
```

### With Grounding (RAG Agent):
```
User: "What's the library closing time on Friday?"
  ↓
Agent: [Uses course_knowledge_query tool]
  ↓
KnowledgeBaseQueryTool: [Calls retriever]
  ↓
SimpleRetriever: [Queries vector store]
  ↓
ChromaDBVectorStore: [Semantic search via embeddings]
  ↓
Returns: "Friday: 7:30 AM - 6:00 PM" from library_hours.txt
  ↓
Agent: [Generates answer based on retrieved text]
  ↓
Agent: "According to the library hours, Kraemer Library closes at 6:00 PM on Fridays."
```

### The RAG Architecture Layers:
```
Agent (Decision Making)
   |
KnowledgeBaseQueryTool (Interface)
   |
SimpleRetriever (Query Logic)
   |
LongTermMemory (Abstraction)
   |
ChromaDBVectorStore (Storage + Search)
   |
SentenceTransformerEmbedder (Vectorization)
```

Each layer has a specific responsibility:
- **Embedder**: Converts text to vectors
- **Vector Store**: Stores and searches embeddings
- **Long-Term Memory**: Provides memory abstraction
- **Retriever**: Implements retrieval logic
- **Tool**: Formats results for the agent
- **Agent**: Makes decisions about when to use the tool

**Key Benefits:**
- ✅ Factual and verifiable
- ✅ Based on YOUR documents
- ✅ Up-to-date (just update the docs!)
- ✅ Transparent (you can see the sources)
- ✅ Modular (swap components as needed)

---

# Part 2: Committees of AI Agents

## Why Use Multiple Agents?

Just like human teams, different agents can have different:
- **Roles** (specialist vs. generalist)
- **Tools** (some agents have access to certain resources)
- **Expertise** (some are better at specific tasks)

### Real-World Examples:
- **Code Review Committee**: One agent writes code, another reviews for bugs, another checks style
- **Essay Grading Team**: One checks grammar, another evaluates argument quality, another verifies facts
- **Research Team**: One searches papers, another summarizes, another synthesizes findings

## Example Task: Essay Evaluation

Let's build a committee to grade essays with three specialized agents:
1. **Grammar Agent**: Checks spelling, grammar, and clarity
2. **Content Agent**: Evaluates argument quality and evidence
3. **Coordinator Agent**: Synthesizes feedback and assigns final grade

## Sample Essay for Testing

In [ ]:
sample_essay = """The Impact of Artificial Intelligence on Education

Artificial intelligence is revolutionizing education in many ways. AI can personalize learning 
by adapting to each students pace and style. For example, intelligent tutoring systems can 
identify where a student struggles and provide targeted practice.

However, their are concerns about AI in education. Some worry that students might become to 
dependent on AI tools and not develop critical thinking skills. Others point out that AI 
systems can perpetuate biases if there trained on biased data.

Despite these challenges, the benefits outweigh the risks. AI can help teachers by automating 
administrative tasks, allowing them to focus on actual teaching. It can also make education 
more accessible to students in remote areas who otherwise wouldn't have access to quality 
instruction.

In conclusion, AI has the potential to transform education for the better, but we must 
implement it thoughtfully and address the ethical concerns. The future of education will 
likely involve a partnership between human teachers and AI systems, combining the best of both.
"""

print("Sample essay loaded!")
print(f"Length: {len(sample_essay.split())} words")

## Building Specialized Agent Tools

First, let's create tools that represent specialized capabilities.

In [ ]:
class GrammarCheckTool(BaseTool):
    """Tool for checking grammar and writing quality"""
    
    name = "check_grammar"
    description = (
        "Analyzes text for grammar, spelling, and clarity issues. "
        "Input: essay text. Returns: detailed grammar feedback."
    )
    
    def use(self, tool_input: str) -> str:
        """Simulate grammar checking"""
        issues = []
        
        # Simple checks (real systems use NLP libraries like LanguageTool)
        if "their are" in tool_input.lower():
            issues.append("- Found 'their are' - should be 'there are' (wrong homophone)")
        if "to dependent" in tool_input.lower():
            issues.append("- Found 'to dependent' - should be 'too dependent' (wrong homophone)")
        if "students pace" in tool_input.lower():
            issues.append("- Found 'students pace' - missing apostrophe: 'student's pace'")
        if "there trained" in tool_input.lower():
            issues.append("- Found 'there trained' - should be 'they're trained' (contraction needed)")
        
        if issues:
            return "Grammar Issues Found:\n" + "\n".join(issues) + "\n\nOverall: Multiple homophone and apostrophe errors detected."
        else:
            return "No major grammar issues detected. Writing is clear and correct."


class ContentAnalysisTool(BaseTool):
    """Tool for analyzing argument quality and evidence"""
    
    name = "analyze_content"
    description = (
        "Evaluates the quality of arguments and evidence in an essay. "
        "Input: essay text. Returns: content quality assessment."
    )
    
    def use(self, tool_input: str) -> str:
        """Simulate content analysis"""
        text_lower = tool_input.lower()
        
        # Count evidence markers
        evidence_markers = ["for example", "research shows", "studies indicate", "data suggests"]
        evidence_count = sum(1 for marker in evidence_markers if marker in text_lower)
        
        # Check structure
        has_intro = "introduction" in text_lower or tool_input.strip().startswith(("Artificial", "The", "In"))
        has_conclusion = "conclusion" in text_lower or "in summary" in text_lower
        
        # Check for counterarguments
        has_counterargument = any(word in text_lower for word in ["however", "although", "despite", "concern"])
        
        feedback = "Content Analysis Results:\n\n"
        feedback += f"Structure:\n"
        feedback += f"  - Clear introduction: {'Yes' if has_intro else 'No'}\n"
        feedback += f"  - Clear conclusion: {'Yes' if has_conclusion else 'No'}\n\n"
        feedback += f"Argumentation:\n"
        feedback += f"  - Evidence examples provided: {evidence_count}\n"
        feedback += f"  - Addresses counterarguments: {'Yes' if has_counterargument else 'No'}\n\n"
        
        if evidence_count < 2:
            feedback += "Suggestion: Add more specific examples and evidence to support claims. "
            feedback += "Consider citing research studies or real-world data.\n"
        if has_counterargument:
            feedback += "Strength: Essay acknowledges opposing views, showing critical thinking.\n"
        
        return feedback


# Create the tools
grammar_tool = GrammarCheckTool()
content_tool = ContentAnalysisTool()

print("✓ Specialized tools created!")

## Creating Specialized Agents

Now we'll create three agents with different roles and tools.

In [ ]:
# Agent 1: Grammar Specialist
grammar_llm = HuggingFaceAdapter(model_name="dolphin3-qwen25-3b", auth_token=token)
grammar_registry = ToolRegistry()
grammar_registry.register(grammar_tool)
grammar_executor = ToolExecutor(grammar_registry)
grammar_memory = WorkingMemory()
grammar_planner = SimpleReActPlanner(grammar_llm, grammar_registry)

grammar_planner.prompt_builder.role_definition = RoleDefinition(
    "You are a Grammar Specialist and writing instructor. "
    "Your job is to evaluate the grammar, spelling, and writing clarity of essays. "
    "Use the check_grammar tool to analyze the text, then provide specific feedback. "
    "Rate grammar quality on a scale of 1-10 and explain your rating. "
    "Be constructive and identify both strengths and areas for improvement."
)

grammar_agent = SimpleAgent(
    llm=grammar_llm,
    planner=grammar_planner,
    tool_executor=grammar_executor,
    memory=grammar_memory,
    max_steps=5
)

print("✓ Grammar Agent created")

In [ ]:
# Agent 2: Content Specialist
content_llm = HuggingFaceAdapter(model_name="dolphin3-qwen25-3b", auth_token=token)
content_registry = ToolRegistry()
content_registry.register(content_tool)
content_executor = ToolExecutor(content_registry)
content_memory = WorkingMemory()
content_planner = SimpleReActPlanner(content_llm, content_registry)

content_planner.prompt_builder.role_definition = RoleDefinition(
    "You are a Content Specialist and critical thinking instructor. "
    "Your job is to evaluate the quality of arguments, evidence, and logical structure in essays. "
    "Use the analyze_content tool to assess the text, then provide feedback on argument strength. "
    "Rate content quality on a scale of 1-10 and explain your rating. "
    "Evaluate: thesis clarity, evidence quality, counterargument consideration, and logical flow."
)

content_agent = SimpleAgent(
    llm=content_llm,
    planner=content_planner,
    tool_executor=content_executor,
    memory=content_memory,
    max_steps=5
)

print("✓ Content Agent created")

In [ ]:
# Agent 3: Coordinator (no special tools, just synthesis)
coordinator_llm = HuggingFaceAdapter(model_name="dolphin3-qwen25-3b", auth_token=token)
coordinator_registry = ToolRegistry()  # No tools needed
coordinator_executor = ToolExecutor(coordinator_registry)
coordinator_memory = WorkingMemory()
coordinator_planner = SimpleReActPlanner(coordinator_llm, coordinator_registry)

coordinator_planner.prompt_builder.role_definition = RoleDefinition(
    "You are the Lead Evaluator and Coordinator. "
    "Your job is to synthesize feedback from the Grammar and Content specialists. "
    "Review their assessments carefully, then provide: "
    "1) An overall grade (A, B, C, D, or F) "
    "2) A summary of key strengths "
    "3) The top 3 areas for improvement "
    "4) One actionable next step for the student "
    "Be fair, balanced, and constructive in your evaluation."
)

coordinator_agent = SimpleAgent(
    llm=coordinator_llm,
    planner=coordinator_planner,
    tool_executor=coordinator_executor,
    memory=coordinator_memory,
    max_steps=5
)

print("✓ Coordinator Agent created")

## Running the Committee

Now let's coordinate the agents to evaluate our essay!

In [ ]:
async def evaluate_essay_with_committee(essay_text):
    """Coordinate multiple agents to evaluate an essay"""
    
    print("="*70)
    print(" " * 20 + "ESSAY EVALUATION COMMITTEE")
    print("="*70)
    
    # Step 1: Grammar Agent evaluates
    print("\n[STEP 1: GRAMMAR SPECIALIST ANALYZING...]\n")
    grammar_feedback = await grammar_agent.arun(
        f"Please evaluate the grammar and writing quality of this essay:\n\n{essay_text}"
    )
    print(f"\n{'─'*70}")
    print("GRAMMAR SPECIALIST REPORT:")
    print(f"{'─'*70}")
    print(grammar_feedback)
    
    # Step 2: Content Agent evaluates
    print(f"\n{'='*70}")
    print("\n[STEP 2: CONTENT SPECIALIST ANALYZING...]\n")
    content_feedback = await content_agent.arun(
        f"Please evaluate the content quality and argumentation of this essay:\n\n{essay_text}"
    )
    print(f"\n{'─'*70}")
    print("CONTENT SPECIALIST REPORT:")
    print(f"{'─'*70}")
    print(content_feedback)
    
    # Step 3: Coordinator synthesizes
    print(f"\n{'='*70}")
    print("\n[STEP 3: LEAD EVALUATOR SYNTHESIZING...]\n")
    
    coordinator_prompt = f"""
Please review the following specialist reports and provide a final evaluation:

GRAMMAR SPECIALIST REPORT:
{grammar_feedback}

CONTENT SPECIALIST REPORT:
{content_feedback}

Provide:
1. Overall Grade (A, B, C, D, or F)
2. Summary of Strengths (2-3 points)
3. Key Areas for Improvement (top 3)
4. One Actionable Next Step for the student
"""
    
    final_evaluation = await coordinator_agent.arun(coordinator_prompt)
    
    print(f"\n{'='*70}")
    print(" " * 25 + "FINAL EVALUATION")
    print(f"{'='*70}")
    print(final_evaluation)
    print(f"\n{'='*70}\n")
    
    return final_evaluation

In [ ]:
# Run the committee evaluation!
await evaluate_essay_with_committee(sample_essay)

### Reflection Question 3

**What advantages does the committee approach have over a single agent?**

Think about:
- **Specialization**: Each agent focuses on what it does best
- **Division of labor**: Complex task broken into manageable pieces
- **Checks and balances**: Multiple perspectives reduce bias
- **Transparency**: You can see each agent's reasoning
- **Modularity**: Easy to add/remove/improve individual specialists

## Understanding Multi-Agent Patterns

### Common Committee Structures:

**1. Sequential Pipeline** (what we just built):
```
Essay → Grammar Agent → Content Agent → Coordinator → Final Grade
```

**2. Parallel Processing**:
```
               ┌─→ Grammar Agent ─┐
    Essay ────┤                   ├──→ Coordinator → Result
               └─→ Content Agent ─┘
```
*Both specialists work simultaneously, then coordinator combines*

**3. Hierarchical Structure**:
```
    Manager Agent
         |
    ┌────┴────┬────────┐
  Worker 1  Worker 2  Worker 3
```
*Manager delegates tasks and coordinates*

**4. Debate/Consensus**:
```
Agent A ←→ Agent B ←→ Agent C
     ↓         ↓         ↓
        Consensus Result
```
*Agents discuss and reach agreement*

## Try It Yourself!

Test the committee on your own essay or writing sample:

In [ ]:
your_essay = """
Paste your essay here!
"""

# Uncomment to test:
# await evaluate_essay_with_committee(your_essay)

---

## Key Insights: Multi-Agent Systems

### Why Committees Beat Single Agents:

**Specialization**:
- Each agent masters one skill
- Like human experts, focused agents perform better
- Reduces cognitive load per agent

**Scalability**:
- Add new specialists as needed
- Agents can work in parallel for speed
- Modular architecture is easier to maintain

**Reliability**:
- Multiple perspectives catch more issues
- One agent's weakness is another's strength
- Cross-validation reduces errors

**Transparency**:
- See each agent's reasoning
- Easier to debug and improve
- Clear audit trail

### When to Use Committees:
- ✅ Complex tasks requiring multiple skills
- ✅ Quality-critical applications (grading, review, analysis)
- ✅ Tasks that benefit from different perspectives
- ✅ When you need explainability and transparency

### When NOT to Use Committees:
- ❌ Simple queries (overkill and slower)
- ❌ Extremely time-sensitive tasks (coordination adds latency)
- ❌ Tasks requiring deep context sharing (single agent is easier)

---

## Combining Grounding + Committees

The most powerful systems combine BOTH techniques:

```
Example: Fact-Checked Essay Grader

Grammar Agent (specialist)
     ↓
Content Agent (specialist)
     ↓
Fact-Checker Agent (grounded on knowledge base via RAG)
     ↓
Coordinator (synthesizes all feedback)
     ↓
Final Report with verified facts
```

Each agent is:
- **Grounded** (has access to relevant knowledge via RAG)
- **Specialized** (focuses on one task)
- **Coordinated** (works with other agents)

### Real-World Application Ideas:
1. **Research Assistant**: Search agent (RAG on papers) → Summarizer → Synthesizer
2. **Code Review**: Linter → Security Checker (grounded on CVE database) → Style Reviewer
3. **Customer Support**: Intent Classifier → Knowledge Base Agent (RAG) → Response Generator

## Final Reflection

### 1. How does grounding solve the hallucination problem?
### 2. When would you use RAG vs. fine-tuning a model?
### 3. What are the tradeoffs between single agents and committees?
### 4. Can you think of a real-world task that would benefit from BOTH grounding AND multiple agents?
### 5. How might you apply these techniques to your final project?

## Congratulations!

You've learned advanced agentic AI techniques:

### Grounding Foundation Models:
- ✓ Why LLMs hallucinate
- ✓ How RAG works (Retrieve → Augment → Generate)
- ✓ Using DocumentProcessor to load documents
- ✓ Creating SimpleRetriever for semantic search
- ✓ Building agents with KnowledgeBaseQueryTool
- ✓ Verifiable, source-backed responses

### Multi-Agent Committees:
- ✓ Agent specialization and roles
- ✓ Coordinating multiple agents
- ✓ Sequential and parallel workflows
- ✓ Synthesizing diverse feedback
- ✓ When to use committees vs. single agents

### What's Next?
- Experiment with different committee structures
- Build agents grounded on YOUR data (PDFs, docs, websites)
- Combine techniques for your final projects
- Explore advanced patterns (debate, voting, hierarchies)
- Build production-ready multi-agent systems

---

## Additional Challenges (Optional)

If you want to explore further, try these:

### Challenge 1: Expand the Knowledge Base
Add more UCCS documents (parking info, dining services, athletics) and test the grounded agent.

### Challenge 2: Add a Third Specialist
Create a "Citation Checker" or "Fact Verifier" agent that uses RAG to verify claims in essays.

### Challenge 3: Parallel Evaluation
Modify the committee to run grammar and content agents in parallel using `asyncio.gather()`:
```python
grammar_task, content_task = await asyncio.gather(
    grammar_agent.arun(prompt),
    content_agent.arun(prompt)
)
```

### Challenge 4: Grounded Fact-Checker
Add a fourth agent that uses the knowledge base to verify factual claims in essays.

### Challenge 5: Your Own Multi-Agent System
Design a committee for a task relevant to your final project. Consider:
- What specialists are needed?
- Which agents need RAG access?
- Sequential or parallel coordination?
- How to synthesize results?